# Projeto: Emissão Acústica Petrobras
## Laboratório de Processamento de Sinais - UFRJ
## Autor: Natanael Junior (natmourajr@lps.ufrj.br)

O objetivo deste notebook é o treinamento e análise de um modelo baseado em redes neurais artificiais aplicado aos dados do projeto 

In [1]:
# Carregando dados
import scipy.io as sio
import numpy as np
mat_contents = sio.loadmat('../../Data/dadosCP2.mat')

inputMatrix = mat_contents['inputMatrix']
fieldNames = mat_contents['fieldNames']
freqPower = mat_contents['power']
normalizedfreqPower = mat_contents['normalizedPower']

feature_labels = []

for x1, x2 in enumerate(fieldNames):
    for y1, y2 in enumerate(x2):
        feature_labels = np.append(feature_labels,str(y2[0])) 


target_file = sio.loadmat('../../Data/targetCP2.mat')
target = target_file['targetCP2']

/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/natmourajr/.v

In [2]:
from Functions import models
nn_params = models.NeuralNetworkParams()

nn_model = models.NeuralNetworkModel()
help(nn_model)

Help on NeuralNetworkModel in module Functions.models object:

class NeuralNetworkModel(Base)
 |  Neural Network Model Class
 |  
 |  Method resolution order:
 |      NeuralNetworkModel
 |      Base
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, verbose=False)
 |  
 |  fit(self, inputs, outputs, train_indexes, n_neurons=2, trn_params=None)
 |      Neural Network Fit Function
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Base:
 |  
 |  __repr__(self)
 |  
 |  __str__(self)
 |  
 |  load(self, path='')
 |  
 |  save(self, path='')
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Base:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing

from Functions import models

iris = datasets.load_iris()

inputs = iris.data  
outputs = iris.target

#print inputs.shape
#print outputs.shape

skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(inputs, outputs)

cat_outputs = -np.ones([outputs.shape[0],len(np.unique(outputs))])
for i,j in enumerate(outputs):
    cat_outputs[i,j] = 1

    
nn_models = {}

for ifold, [train_index, test_index] in enumerate(skf.split(inputs, outputs)):
    if ifold > 0:        
        continue
    #normalize data
    scaler = preprocessing.StandardScaler().fit(inputs[train_index,:])
    normalized_inputs = scaler.transform(inputs)
    
    nn_models[ifold] = models.NeuralNetworkModel()
    
    nn_models[ifold].fit(inputs=normalized_inputs, 
                         outputs=cat_outputs,
                         train_indexes=[train_index, test_index],
                         n_neurons=10,)

/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/home/natmourajr/.virtualenvs/petro_project/local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 9

In [6]:
nn_models[0].model.layers